In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import itertools

In [2]:
names = ['mobile','touchpoint_id','action_time','row_number','time_lag','tp_id_through','row_number_lag','conv_period']
df = pd.read_csv('data/tp_analysis_base.csv',sep = '\t', names = names)
df['mobile'] = df['mobile'].astype(str)

In [3]:
id_mapping = pd.read_csv('data/id_mapping.csv')

tp_name = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.touchpoint_name)}
inbound_map = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.is_inbound)}

In [4]:
# fetch other information
deal_date = pd.read_csv('data/deal_time.csv',names=['mobile','deal_time'],sep='\t')

In [5]:
deal_date_g = deal_date.groupby(by='mobile',as_index=False).agg({'deal_time':'min'})

In [6]:
old_deal = deal_date[deal_date.deal_time <= '2019-06-01']
new_deal = deal_date[deal_date.deal_time > '2019-06-01']

In [7]:
cust_user = df[(df.touchpoint_id == '004000000000_tp') | (df.touchpoint_id == '005000000000_tp')].mobile.unique().tolist()

In [8]:
# segment
df['segment'] = np.nan
df['segment'][df.mobile.isin(old_deal.mobile.unique().tolist())] = 'old_deal'
df['segment'][df.mobile.isin(new_deal.mobile.unique().tolist())] = 'new_deal'
df['segment'][(df.mobile.isin(cust_user)) & (df.segment.isna()==True)] = 'cust'
df['segment'] = df['segment'].fillna('potential')

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
df[['mobile','segment']].drop_duplicates().segment.value_counts()

potential    1569281
cust         1425932
old_deal      403028
new_deal      189750
Name: segment, dtype: int64

In [42]:
df[df.touchpoint_id.isin(id_mapping[id_mapping.data_source == 'app'].touchpoint_id.tolist())][['mobile','segment']].drop_duplicates().segment.value_counts()

potential    352303
new_deal     128621
old_deal      95904
cust          84088
Name: segment, dtype: int64

In [23]:
app_id = [i for i in id_mapping.touchpoint_id.tolist() if i.startswith('003')] + \
[i for i in id_mapping.touchpoint_id.tolist() if i.startswith('008002')] + \
[i for i in id_mapping.touchpoint_id.tolist() if i.startswith('002') and i.startswith('002011') == False] +\
[i for i in id_mapping.touchpoint_id.tolist() if i.startswith('019')] 

app_user = df[df.touchpoint_id.isin(app_id)].groupby(by='mobile',as_index=False).agg({'action_time':'min'})

# 意向建卡
cust_base = df[(df.touchpoint_id == '004000000000_tp') | (df.touchpoint_id == '005000000000_tp')][['mobile','action_time']]\
.rename(columns={'action_time':'cust_time'})\
.groupby(by='mobile',as_index=False).agg({'cust_time':'min'})

visit_df_g = visit_df.groupby(by='mobile',as_index=False).agg({'visit_time':'min'})
deal_df_g = deal_df.groupby(by='mobile',as_index=False).agg({'deal_time':'min'})
trial_df_g = trial_df.groupby(by='mobile',as_index=False).agg({'trial_time':'min'})

In [62]:
app_cust = app_user.merge(cust_base[['mobile']],on='mobile',how='inner').drop_duplicates()
app_cust = app_cust.merge(visit_df_g,on='mobile',how='left').merge(trial_df_g,on='mobile',how='left').merge(deal_df_g,on='mobile',how='left')

In [28]:
register_df = df[(df.touchpoint_id == '002001002000_tp')][['mobile','action_time']]\
.rename(columns={'action_time':'regis_time'})\
.groupby(by='mobile',as_index=False).agg({'regis_time':'min'})

In [232]:
app_sour = pd.read_csv('data/MGAPP_original_source_with_mobile.csv').dropna()
app_sour.cellphone = app_sour.cellphone.astype(int).astype(str)

app_sour = app_sour.merge(cust_base,left_on = 'cellphone',right_on='mobile',how='left').drop(columns='mobile')
app_sour = app_sour.merge(app_user,left_on = 'cellphone',right_on='mobile',how='left').drop(columns='mobile')
app_sour = app_sour.merge(register_df,left_on = 'cellphone',right_on='mobile',how='left').drop(columns='mobile')
app_sour = app_sour.merge(deal_date_g,left_on = 'cellphone',right_on='mobile',how='left').drop(columns='mobile')
app_sour = app_sour.merge(trial_df,left_on = 'cellphone',right_on='mobile',how='left').drop(columns='mobile')
app_sour = app_sour.merge(visit_df,left_on = 'cellphone',right_on='mobile',how='left').drop(columns='mobile')

In [234]:
app_sour['start_time'] = [i[0] if type(i[0]) != float else i[1] for i in zip(app_sour.action_time,app_sour.regis_time)]

In [235]:
deal_list = app_sour[app_sour.start_time > app_sour.deal_time][['cellphone']]
deal_list['segment'] = 'deal'
cust_list = app_sour[(app_sour.start_time > app_sour.cust_time) & (app_sour.cellphone.isin(deal_list.cellphone.tolist()) == False)][['cellphone']]
cust_list['segment'] = 'cust'

In [236]:
app_sour = app_sour.merge(cust_list.append(deal_list).drop_duplicates(),on='cellphone',how='left')
app_sour['segment'] = app_sour['segment'].fillna('potential')

In [239]:
# 各渠道用户数
df_res = pd.pivot_table(data=app_sour,index='campaign_name',values='cellphone',aggfunc='count',columns='segment')

In [245]:
# 首次使用APP后到店，试驾，成交
df_res.merge(pd.pivot_table(data=app_sour[app_sour.trial_time >= app_sour.start_time],index='campaign_name',
                            values='cellphone',aggfunc='count',columns='segment'),
             left_index=True, right_index=True, how='left')\
.merge(pd.pivot_table(data=app_sour[app_sour.visit_time >= app_sour.start_time],index='campaign_name',
                      values='cellphone',aggfunc='count',columns='segment'),
       left_index=True, right_index=True, how='left')\
.merge(pd.pivot_table(data=app_sour[app_sour.deal_time >= app_sour.start_time],index='campaign_name',
                      values='cellphone',aggfunc='count',columns='segment'),
              left_index=True, right_index=True, how='left')